In [1]:
import ccxt
import config_API_Bybit as cab
from telegram import Bot
import asyncio

In [2]:
# Inizializza il bot di Telegram
telegram_bot = Bot(token=cab.token_telegram)
TOKEN_TELEGRAM = cab.token_telegram
CHAT_ID_TELEGRAM = cab.id_chat_mio_gruppo

In [3]:
# Inizializza l'exchange Bybit
exchange = ccxt.bybit({
    'apiKey': cab.api_key,
    'secret': cab.api_secret,
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True,
    },
})

# Disabilita la modalità sandbox per l'ambiente live
exchange.set_sandbox_mode(False)

# Carica i mercati
exchange.load_markets()

def get_balance(crypto_symbol, account_type):
    """
    Recupera il saldo di una specifica criptovaluta in un determinato account.

    :param crypto_symbol: Simbolo della criptovaluta (es. 'FOXY')
    :param account_type: Tipo di account (es. 'spot', 'funding', 'unified')
    :return: Dizionario con i saldi 'free', 'used' e 'total'
    """
    try:
        # Recupera il saldo per il tipo di account specificato
        balance = exchange.fetch_balance({'type': account_type, 'recvWindow': 20000}) # 10 secondi
        # Ottieni il saldo della criptovaluta specificata
        crypto_balance = balance.get(crypto_symbol, {'free': 0, 'used': 0, 'total': 0})
        return crypto_balance
    except Exception as e:
        print(f"Errore durante il recupero del saldo per {crypto_symbol} nell'account {account_type}: {e}")
        return None

# Esempio di utilizzo
crypto = 'USDT'
account_types = ['spot', 'funding', 'unified']

for account in account_types:
    balance = get_balance(crypto, account)
    if balance:
        print(f"Saldo di {crypto} nell'account {account}:")
        print(f"  Disponibile: {balance['free']}")
        print(f"  In uso: {balance['used']}")
        print(f"  Totale: {balance['total']}\n")

InvalidNonce: bybit {"retCode":10002,"retMsg":"invalid request, please check your server timestamp or recv_window param. req_timestamp[1734722778668],server_timestamp[1734722777548],recv_window[5000]","result":{},"retExtInfo":{},"time":1734722777548}

In [3]:
def invia_messaggio_telegram(token, chat_id, messaggio):
    try:
        async def _invia():
            bot = Bot(token=token)
            await bot.send_message(chat_id=chat_id, text=messaggio)
        
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(_invia())
        else:
            loop.run_until_complete(_invia())
    except Exception as e:
        print(f"Errore nell'invio del messaggio Telegram: {e}")

invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, 'ciao di prova')

In [5]:
def buy_crypto(symbol, amount, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)  # Imposta a True per il testnet
    exchange.load_markets()
    order = exchange.create_market_buy_order(symbol, amount)
    return order

In [6]:
def sell_crypto(symbol, amount, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    order = exchange.create_market_sell_order(symbol, amount)
    return order

In [7]:
def set_trailing_stop(symbol, side, amount, trailing_stop, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    params = {
        'stop_loss': trailing_stop,
    }
    order = exchange.create_order(symbol, 'market', side, amount, None, params)
    return order

In [8]:
def get_average_buy_price(symbol, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    orders = exchange.fetch_closed_orders(symbol)
    total_cost = 0
    total_amount = 0
    for order in orders:
        if order['side'] == 'buy':
            total_cost += order['cost']
            total_amount += order['filled']
    if total_amount > 0:
        return total_cost / total_amount
    else:
        return 0

In [9]:
def cancel_trailing_stop_orders(symbol, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    orders = exchange.fetch_open_orders(symbol)
    for order in orders:
        if 'trailing' in order['info']:
            exchange.cancel_order(order['id'], symbol)

In [10]:
def get_available_usdt(api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    balance = exchange.fetch_balance()
    return balance['USDT']['free']

In [ ]:
def execute_trade():
    try:
        # Esegui operazioni di trading
        order = buy_crypto('BTC/USDT', 0.01, cab.api_key, cab.api_secret)
        # Altre operazioni...
    except Exception as e:
        error_message = f"Errore durante l'esecuzione del trade: {str(e)}"
        invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, error_message)

In [ ]:
# invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, 'Altra prova')

# Codice di ChatGPT o1

In [ ]:
import ccxt
import config_API_Bybit as cab
import math
import asyncio
import time

In [ ]:
api_key = 'LA_TUA_API_KEY'
api_secret = 'LA_TUA_API_SECRET'
token_telegram = 'IL_TOKEN_DEL_BOT_TELEGRAM'
id_chat_mio_gruppo = 'LA_TUA_CHAT_ID'

In [ ]:
################################################################################
# 1) Funzione per creare l'exchange Bybit già configurato
################################################################################
def get_bybit_exchange(api_key=cab.api_key, 
                       api_secret=cab.api_secret, 
                       sandbox=False):
    """
    Restituisce un oggetto ccxt.bybit configurato con le chiavi API.
    """
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
        'options': {
            'adjustForTimeDifference': True,
        },
    })
    # Setta sandbox a True/False
    exchange.set_sandbox_mode(sandbox)

    # Carica i mercati
    exchange.load_markets()
    return exchange

In [ ]:
################################################################################
# 2) Funzioni di base per Spot Trading
################################################################################

def get_balance(exchange, crypto_symbol, account_type='spot'):
    """
    Recupera il saldo di una specifica criptovaluta in un determinato account.
    :param exchange: l'oggetto ccxt.bybit (già configurato)
    :param crypto_symbol: simbolo, es. 'USDT'
    :param account_type: 'spot', 'funding', 'unified', ecc.
    :return: dizionario con i saldi { 'free': float, 'used': float, 'total': float }
    """
    try:
        balance = exchange.fetch_balance({'type': account_type, 'recvWindow': 20000})
        crypto_balance = balance.get(crypto_symbol, {'free': 0, 'used': 0, 'total': 0})
        return crypto_balance
    except Exception as e:
        print(f"Errore durante il recupero del saldo per {crypto_symbol} nell'account {account_type}: {e}")
        return None


def buy_crypto_market(exchange, symbol, amount):
    """
    Esegue un ordine di acquisto MARKET su Bybit (Spot).
    :param exchange: oggetto ccxt.bybit
    :param symbol: es. 'BTC/USDT'
    :param amount: quantità di BTC da acquistare
    """
    try:
        order = exchange.create_market_buy_order(symbol, amount)
        return order
    except Exception as e:
        print(f"Errore buy_crypto_market su {symbol}: {e}")
        return None


def sell_crypto_market(exchange, symbol, amount):
    """
    Esegue un ordine di vendita MARKET su Bybit (Spot).
    :param exchange: oggetto ccxt.bybit
    :param symbol: es. 'BTC/USDT'
    :param amount: quantità di BTC da vendere
    """
    try:
        order = exchange.create_market_sell_order(symbol, amount)
        return order
    except Exception as e:
        print(f"Errore sell_crypto_market su {symbol}: {e}")
        return None


def set_trailing_stop_spot(exchange, symbol, side, amount, trigger_price, trail_offset):
    """
    ESEMPIO TEORICO per piazzare un trailing stop su SPOT Bybit.
    Su Bybit Spot potrebbe non essere supportato nativamente.
    Questo snippet è da testare ed eventualmente adattare. Spesso
    è necessario fare uno "stop-limit" e aggiornarlo in loop.
    """
    # Se side == 'buy' => piazzi uno stop-limit buy sopra il prezzo
    # Se side == 'sell' => piazzi uno stop-limit sell sotto il prezzo
    # 'trigger_price' è il prezzo di attivazione
    # 'trail_offset' potrebbe essere interpretato come differenza dal trigger
    try:
        params = {
            # Dipende dal tipo di ordine che Bybit accetta su Spot
            # In alcuni casi:
            # 'stopPrice': trigger_price,
            # 'timeInForce': 'GTC',
            # ...
        }
        # A volte su ccxt Bybit Spot si fa: create_order(symbol, type, side, amount, price, params)
        # Proviamo con uno stop-limit (fittizio).
        order_price = trigger_price  # es. un prezzo peggiore/migliore del mercato
        order = exchange.create_order(
            symbol, 
            'stop_limit',  
            side, 
            amount, 
            order_price, 
            params
        )
        return order
    except Exception as e:
        print(f"Errore set_trailing_stop_spot su {symbol}: {e}")
        return None


def cancel_all_open_orders(exchange, symbol):
    """
    Cancella tutti gli ordini aperti su uno specifico simbolo.
    """
    try:
        orders = exchange.fetch_open_orders(symbol)
        for o in orders:
            order_id = o['id']
            exchange.cancel_order(order_id, symbol)
        return True
    except Exception as e:
        print(f"Errore durante la cancellazione degli ordini aperti su {symbol}: {e}")
        return False


def fetch_trade_history(exchange, symbol, limit=50):
    """
    Recupera la trade history (ordini eseguiti).
    """
    try:
        return exchange.fetch_my_trades(symbol, limit=limit)
    except Exception as e:
        print(f"Errore durante il recupero della trade history su {symbol}: {e}")
        return []

In [ ]:
################################################################################
# 3) Logica di trading a “ciclo completo”
################################################################################

def trading_cycle():
    """
    Esegue un ciclo completo di trading sul pair BTC/USDT in Spot:
    1. Imposta un trailing stop buy (senza prezzo attivazione) per iniziare.
    2. Quando scatta l'acquisto:
       - Imposta trailing buy + trailing sell con attivazione.
       - Cancella eventuali ordini obsoleti.
    3. Quando scatta un altro acquisto:
       - Cancella ordini rimanenti.
       - Rimposta trailing buy + trailing sell con attivazione.
    4. Quando finalmente avviene la vendita, finisce il ciclo e ricomincia.
    """
    symbol = 'BTC/USDT'
    amount_to_buy = 0.001  # Esempio
    exchange = get_bybit_exchange()  # crea exchange live (o sandbox se vuoi)
    
    # 1. Imposta un (finto) trailing stop buy
    # Nota: su Bybit Spot non c'è un trailing stop buy "nativo" -> si simula
    # Ad esempio, creiamo uno STOP-LIMIT BUY a un prezzo X
    current_ticker = exchange.fetch_ticker(symbol)
    current_price = current_ticker['last']
    print(f"Prezzo attuale di {symbol}: {current_price}")

    # Esempio di prezzo di trigger "un po' sopra" l'attuale
    trigger_price = current_price * 1.01  # 1% sopra l'attuale
    # Esempio offset
    trail_offset = 0.005  # 0.5%
    # Creiamo l'ordine (fittizio, in realta' e' uno stop-limit)
    # Avrai bisogno di testare come Bybit si comporta
    trailing_buy_order = set_trailing_stop_spot(
        exchange, 
        symbol, 
        'buy', 
        amount_to_buy, 
        trigger_price, 
        trail_offset
    )
    print("Trailing buy (finto) creato:", trailing_buy_order)

    # In un mondo ideale, qui attendiamo che l'ordine venga eseguito
    # (monitoriamo ciclicamente lo stato dell'ordine).
    # Se eseguito => scatta "on_buy_filled()"
    # Se cancellato => scatta "on_buy_cancelled()" e si ricomincia
    # ...
    # Per esempio:
    while True:
        # Verifica se l'ordine è eseguito
        open_orders = exchange.fetch_open_orders(symbol)
        still_open = any(o['id'] == trailing_buy_order['id'] for o in open_orders if trailing_buy_order)
        if not still_open:
            # Ordine eseguito o cancellato
            # Controlliamo se eseguito (fetch order info)
            order_info = exchange.fetch_order(trailing_buy_order['id'], symbol)
            if order_info['status'] == 'closed':
                print("Ordine buy eseguito!")
                # 2. Impostiamo trailing buy + trailing sell con attivazione
                # (esempio)
                cancel_all_open_orders(exchange, symbol)
                # Qui setti i tuoi parametri di trailing in base alla strategia
                # ...
                # Se poi scatta un nuovo buy, rifai la cancellazione e reimposti i trailing
                # ...
                # Poi attendi la vendita. Se scatta la vendita, il ciclo finisce
                print("Fine del ciclo, ricomincio...")
                break
            else:
                print("Ordine buy cancellato/manualmente chiuso. Riprovo.")
                break
        else:
            print("Ordine buy ancora aperto, attendo 5 secondi...")
            time.sleep(5)

In [ ]:
trades = fetch_trade_history(exchange, 'BTC/USDT', limit=10)
for t in trades:
    print(f"Trade {t['id']}: side={t['side']}, qty={t['amount']}, price={t['price']}")